In [1]:
%pip install -q pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
def next_ref(refs: dict) -> tuple[int, str]:
    """
    Get the next reference number and reference string.

    Args:
        refs (dict): A dictionary of references.

    Returns:
        tuple[int, str]: The next reference number and reference string.
    """
    next_num = max(refs.keys()) + 1
    next_ref = f'ref_{str(next_num).zfill(6)}'
    return next_num, next_ref


In [3]:
from typing import List

def transform_dict_for_homebox(data: List[dict]) -> List[dict]:
    """
    Transform a list of dictionaries to a list of dictionaries with keys
    that are compatible with HomeBox.
    """
    keymap = {
        'name': 'HB.name',
        'category': 'HB.field.category',
        'qty': 'HB.quantity',
        'location': 'HB.location',
        'ref': 'HB.import_ref',
    }

    # Reserve the first 5 reference numbers for HomeBox
    ref_nums: dict = {
        0: True,
        1: True,
        2: True,
        3: True,
        4: True,
    }

    transformed_data = []
    for item in data:
        transformed_item = {}
        for k, v in item.items():
            transformed_item[keymap[k]] = v
        ref_n, ref_s = next_ref(ref_nums)
        transformed_item['HB.import_ref'] = ref_s
        transformed_data.append(transformed_item)
        ref_nums[ref_n] = True
        
    return transformed_data

In [4]:
import csv

# Load HomeBox CSV
with open('csv_inventory.csv', mode='r') as file:
    reader = csv.DictReader(file)
    homebox_data = [row for row in reader]

transformed_data = transform_dict_for_homebox(homebox_data)

# Write transformed data to a new CSV file
with open('homebox_inventory.csv', mode='w') as file:
    writer = csv.DictWriter(file, fieldnames=transformed_data[0].keys())
    writer.writeheader()
    for row in transformed_data:
        writer.writerow(row)